## **This notebook aims to compare some models and store them in a reasonable fashion**

In [2]:
import pandas as pd
import numpy as np
import os
import time
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU 0 in first notebook

In [4]:
project_dir = "/home/jupyter-tfg2425paula/prediction_project_v3"
os.chdir(project_dir)

clean_data_dir = os.path.join(project_dir, "00_data/clean")
horizontal_data_dir = os.path.join(project_dir, "00_data/horizontal_structure")
results_dir = os.path.join(project_dir, "02_results")
plots_dir = os.path.join(project_dir, "03_plots")
pca_data_dir = os.path.join(project_dir, "00_data/pca")

### **GRU Model**

In [5]:
class GRU3DClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(GRU3DClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :]) 
        # return self.sigmoid(out)
        return out

### **LSTM Model**

In [6]:
class StockPriceLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, dropout=0.0):
        super(StockPriceLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, 
                            batch_first=True, dropout=dropout)
    
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        batch_size = x.size(0)  # Get the batch size dynamically

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)  # (num_layers, batch_size, hidden_dim)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)  # (num_layers, batch_size, hidden_dim)
        
        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :]) 
        # out = self.sigmoid(out)
        return out
    

### **Set folders**

Processing

In [7]:
processing_types = ["clean", "pca"]
processing_types= ["clean"]

Folders

In [8]:
# stocks = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'SPX']
stocks = ['AAPL']
# types_securities = ["single_name", "options", "technical"]
types_securities = ["options"]

Different files

In [9]:
# years = ["15y", "10y", "5y", "2y"]
years = ["10y"]
# window_sizes = [5, 10, 50, 100]
window_sizes = [5]
# train_sizes = [80, 90, 95]
train_sizes = [95]

Same file

In [10]:
thresholds = [0.3, 0.35, 0.4, 0.45, 0.5]
thresholds = [0.5]
learning_rates = [0.005, 0.008, 0.009, 0.01]
learning_rates = [0.01]
num_epochs_list = [100, 200]
num_epochs_list = [100]
batch_sizes = [16, 32]
batch_sizes = [16]
prediction_thresholds = [0.35, 0.4, 0.45, 0.5]
prediction_thresholds = [0.5]

#### **Model and Hyperparameters**

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hidden_size = 64  
output_size = 2  
num_layers = 2
dropout = 0.2

criterion = nn.CrossEntropyLoss()

In [12]:
model_types = ["lstm", "gru"]   

#### **Last data modifications**

In [13]:
def reshape_remove_characters(df):

    X = np.array([np.stack(row) for row in df.drop(columns=['Target']).values])
    y = df['Target'].values

    smote = SMOTE(random_state=42)
    n_samples, timesteps, n_features = X.shape
    X_flat = X.reshape((n_samples, timesteps * n_features))
    X_flat = np.where(X_flat == 'ç', 0, X_flat)

    X_resampled = X_flat.reshape((-1, timesteps, n_features))
    
    return X_resampled, y

### **Evaluation function**

In [14]:
def evaluate_rolling_unchanged_model_threshold(
    model, 
    X, 
    y, 
    criterion, 
    optimizer, 
    device, 
    train_size, 
    batch_size, 
    num_epochs, 
    lower_threshold
):
    """
    Evaluate a PyTorch model using a rolling prediction approach for time series,
    training the model only once on the initial training set. For each time step
    after train_size, the model makes a prediction without further parameter updates.
    Only predicts +1 or -1 if the probability of class 1 is above/below given thresholds;
    otherwise, predicts 0. Accuracy is computed only on nonzero predictions.

    Args:
        model:          PyTorch model to evaluate.
        X:              Feature data (numpy array).
        y:              Target data (numpy array).
        criterion:      Loss function (e.g., CrossEntropyLoss).
        optimizer:      Optimizer (e.g., Adam).
        device:         Device for computation (CPU or GPU).
        train_size:     Initial size of the training data (int or float).
                        If < 1, treated as fraction of total length.
        batch_size:     Batch size for training.
        num_epochs:     Number of epochs for initial training only.
        lower_threshold: Probability threshold below which model predicts -1.
        upper_threshold: Probability threshold above which model predicts +1.

    Returns:
        dict: Dictionary with the following keys:
            - "rolling_predictions": All predictions (-1, 0, +1) across the test period.
            - "rolling_targets": Corresponding true targets in [-1, +1].
            - "filtered_predictions": Nonzero predictions only.
            - "filtered_targets": Targets corresponding to nonzero predictions.
            - "accuracy_nonzero": Accuracy computed only on nonzero predictions.
    """

    # Convert X, y to tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    # Determine initial training set size
    if train_size < 1.0:
        lower_bound = int(train_size * len(X))
    else:
        lower_bound = train_size

    # -------------------------
    # 1) SINGLE TRAINING PHASE
    # -------------------------
    model.to(device)
    model.train()
    
    X_train = X[:lower_bound].to(device)
    y_train = y[:lower_bound].to(device)

    train_dataset = TensorDataset(X_train, y_train)
    trainloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=False,         # Keep False if order matters; True for better generalization
        # num_workers=4,         # Adjust based on your CPU cores
        # pin_memory=True,       # Speeds up transfer if using GPUs
        drop_last=False        # Ensure the last batch is included
    )

    epoch_train_losses = []
    for epoch in range(num_epochs):
        # torch.cuda.empty_cache()
        epoch_loss = 0.0
        for X_batch, y_batch in trainloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            pred_y = model(X_batch)   # [batch_size, num_classes]
            loss = criterion(pred_y, y_batch)
            loss.backward()

            # Gradient clipping (optional)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()
               
        if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
            print(f"[Train] Epoch {epoch+1}/{num_epochs}, Loss={epoch_loss/len(trainloader):.4f}")

        epoch_train_losses.append(epoch_loss/len(trainloader))
        
    loss_decrease_percentage = ((epoch_train_losses[-1] - epoch_train_losses[0]) / epoch_train_losses[0]) * 100
    # ---------------------------------
    # 2) ROLLING PREDICTIONS, NO UPDATE
    # ---------------------------------
    model.eval()

    rolling_predictions = []
    rolling_targets     = []

    for i in range(lower_bound, len(X)):
        # Single-step "test" sample
        X_test = X[i:i+1].to(device)  # shape: (1, num_features)
        y_test = y[i:i+1].to(device)  # shape: (1, )

        with torch.no_grad():
            # Forward pass
            pred_y = model(X_test)  # [1, num_classes]
            probabilities = torch.softmax(pred_y, dim=1).cpu().numpy()  # shape: (1, 2)
            prob_class_1  = probabilities[:, 1]  # shape: (1,)

            # Threshold-based logic
            # Initialize all predictions to 0
            pred_classes = np.zeros_like(prob_class_1)
            # Predict -1 if prob < lower_threshold
            pred_classes[prob_class_1 < lower_threshold] = -1
            # Predict +1 if prob > upper_threshold
            pred_classes[prob_class_1 > 1-lower_threshold] = 1

        rolling_predictions.append(pred_classes[0])  # scalar
        rolling_targets.append(y_test.item())

    rolling_predictions = np.array(rolling_predictions)
    rolling_targets = np.array(rolling_targets).astype(int)

    # Convert any 0-labeled targets to -1 if your original data is in [-1, +1]
    # (Sometimes y might be {0,1} or {-1, +1}; adapt as needed.)
    rolling_targets[rolling_targets == 0] = -1

    # Filter out zero predictions
    nonzero_mask = rolling_predictions != 0
    filtered_preds = rolling_predictions[nonzero_mask]
    filtered_targets = rolling_targets[nonzero_mask]

    if len(filtered_preds) == 0:
        accuracy_nonzero = None
        print("No nonzero predictions, cannot compute thresholded accuracy.")
    else:
        accuracy_nonzero = accuracy_score(filtered_targets, filtered_preds)
        print(f"Accuracy on Nonzero Predictions: {accuracy_nonzero:.4f}")

    return {
        "rolling_predictions": rolling_predictions,
        "rolling_targets": rolling_targets,
        "filtered_predictions": filtered_preds,
        "filtered_targets": filtered_targets,
        "accuracy_nonzero": accuracy_nonzero,
        "loss_decrease_percentage": loss_decrease_percentage,
        "final_train_loss": epoch_train_losses[-1] 
    }

### **3nd Type of comparison:**

Learning rates, for AAPL 10y

### **Plot curves**

In [15]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

def evaluate_rolling_unchanged_model_threshold(
    model, 
    X, 
    y, 
    criterion, 
    optimizer, 
    device, 
    train_size, 
    batch_size, 
    num_epochs, 
    lower_threshold,
    plots_dir=None,
    plot_filename=None
):

    # -------------------------------
    # 0) Prepare Tensors & Splits
    # -------------------------------
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    total_size = len(X)
    # Determine actual train_size index
    if train_size < 1.0:
        lower_bound = int(train_size * total_size)
    else:
        lower_bound = train_size

    # Training portion
    X_train = X[:lower_bound].to(device)
    y_train = y[:lower_bound].to(device)

    train_dataset = TensorDataset(X_train, y_train)
    trainloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=False,   # Set True if you prefer shuffling
        drop_last=False
    )

    if lower_bound < total_size:
        X_val = X[lower_bound:].to(device)
        y_val = y[lower_bound:].to(device)
        val_dataset = TensorDataset(X_val, y_val)
        valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    else:
        # If there's no leftover data for "test", handle gracefully
        X_val = None
        y_val = None
        valloader = None

    # ---------------------------------------------
    # 1) SINGLE TRAINING PHASE + Track Loss Curves
    # ---------------------------------------------
    model.to(device)
    model.train()

    epoch_train_losses = []
    epoch_test_losses = []

    for epoch in range(num_epochs):
        # TRAINING PASS
        epoch_loss = 0.0
        for X_batch, y_batch in trainloader:
            optimizer.zero_grad()

            # Forward
            pred_y = model(X_batch)
            loss = criterion(pred_y, y_batch)

            # Backprop & update
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # optional
            optimizer.step()

            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(trainloader)
        epoch_train_losses.append(avg_train_loss)

        # VALIDATION PASS (Optional but needed to get test_loss_curve)
        if valloader is not None:
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for Xb, yb in valloader:
                    pred_yb = model(Xb)
                    loss_b = criterion(pred_yb, yb)
                    val_loss += loss_b.item()
            avg_val_loss = val_loss / len(valloader)
            epoch_test_losses.append(avg_val_loss)

            model.train()  # Switch back to train mode

        else:
            # If no validation set, just store None or 0
            epoch_test_losses.append(None)

        # Print progress every 5 epochs or last epoch
        if (epoch + 1) % 5 == 0 or (epoch == num_epochs - 1):
            if epoch_test_losses[-1] is not None:
                print(f"[Epoch {epoch+1}/{num_epochs}] "
                      f"Train Loss: {avg_train_loss:.4f}, "
                      f"Val Loss: {avg_val_loss:.4f}")
            else:
                print(f"[Epoch {epoch+1}/{num_epochs}] "
                      f"Train Loss: {avg_train_loss:.4f}")

    # % decrease from first to last train loss
    if len(epoch_train_losses) > 1:
        loss_decrease_percentage = ((epoch_train_losses[-1] - epoch_train_losses[0])
                                    / epoch_train_losses[0]) * 100
    else:
        loss_decrease_percentage = 0.0

    final_train_loss = epoch_train_losses[-1]

    # -------------------------------
    # 2) ROLLING PREDICTIONS, NO UPDATE
    # -------------------------------
    model.eval()
    rolling_predictions = []
    rolling_targets = []

    for i in range(lower_bound, total_size):
        X_test = X[i:i+1].to(device)
        y_test = y[i:i+1].to(device)

        with torch.no_grad():
            pred_y = model(X_test)
            probabilities = torch.softmax(pred_y, dim=1).cpu().numpy()
            prob_class_1 = probabilities[:, 1]  # shape: (1,)

            # Threshold-based logic
            pred_classes = np.zeros_like(prob_class_1)
            # Predict -1 if prob < lower_threshold
            pred_classes[prob_class_1 < lower_threshold] = -1
            # Predict +1 if prob > (1 - lower_threshold)
            pred_classes[prob_class_1 > (1 - lower_threshold)] = 1

        rolling_predictions.append(pred_classes[0])
        rolling_targets.append(y_test.item())

    rolling_predictions = np.array(rolling_predictions)
    rolling_targets = np.array(rolling_targets).astype(int)

    # If original labels might be {0,1}, adapt as needed
    rolling_targets[rolling_targets == 0] = -1

    # Filter out zero predictions
    nonzero_mask = (rolling_predictions != 0)
    filtered_preds = rolling_predictions[nonzero_mask]
    filtered_targets = rolling_targets[nonzero_mask]

    if len(filtered_preds) == 0:
        accuracy_nonzero = None
        print("No nonzero predictions, cannot compute thresholded accuracy.")
    else:
        accuracy_nonzero = accuracy_score(filtered_targets, filtered_preds)
        print(f"Accuracy on Nonzero Predictions: {accuracy_nonzero:.4f}")

    # -------------------------------------------------
    # 3) PLOT (if plots_dir is set and there's test data)
    # -------------------------------------------------
    if plots_dir is not None:
        os.makedirs(plots_dir, exist_ok=True)

        # If user didn't provide a filename, create a default
        if plot_filename is None:
            plot_filename = "train_test_loss_curve.png"
        plot_path = os.path.join(plots_dir, plot_filename)

        # Plot the training and validation (test) loss curves
        plt.figure(figsize=(8, 5))
        plt.plot(epoch_train_losses, label="Train Loss")
        # Only plot test loss if it isn't None
        if any(x is not None for x in epoch_test_losses):
            plt.plot(epoch_test_losses, label="Test Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Train vs. Test Loss per Epoch")
        plt.legend()
        plt.tight_layout()

        plt.savefig(plot_path)
        plt.close()
        print(f"Loss curves saved to: {plot_path}")

    # ----------------
    # 4) Return results
    # ----------------
    return {
        "rolling_predictions": rolling_predictions,
        "rolling_targets": rolling_targets,
        "filtered_predictions": filtered_preds,
        "filtered_targets": filtered_targets,
        "accuracy_nonzero": accuracy_nonzero,
        "loss_decrease_percentage": loss_decrease_percentage,
        "final_train_loss": final_train_loss,
        "train_loss_curve": epoch_train_losses,
        "test_loss_curve": epoch_test_losses
    }


In [26]:
results_list = []
stock = "AAPL"
period = "10y"
possible_train_size = 95
batch_size = 32
num_epochs = 80
window_size = 3

prediction_threshold = 0.5

output_folder = os.path.join(results_dir, f"individual_trials") 
os.makedirs(output_folder, exist_ok=True)

processing = "clean"
security_types = ["single_name"]
learning_rate = 0.009883203468677031
model_type = "gru"

results_list = []
for security_type in security_types:
    initial_data_dir = os.path.join(project_dir, f"00_data/{processing}") 
    
    # 1) Load original data (info only)
    filename = f"{security_type}/{stock}/{period}_data.csv"
    original_input_filepath = os.path.join(initial_data_dir, filename)
    original_data = pd.read_csv(original_input_filepath)

    print(f"\n----- LEARNING_RATE: {learning_rate}, SECURITY_TYPE: {security_type}, MODEL_TYPE: {model_type} -----")

    # 2) Load the preprocessed data
    pkl_filename = f"{processing}/{security_type}/{stock}/{period}_{window_size}_data.pkl"
    input_filepath = os.path.join(horizontal_data_dir, pkl_filename)
    input_df = pd.read_pickle(input_filepath)
    
    # 3) Reshape
    X_resampled, y_resampled = reshape_remove_characters(input_df)

    input_size = X_resampled.shape[2]
    train_size = int(X_resampled.shape[0] * possible_train_size / 100)
    test_size = X_resampled.shape[0] - train_size

    for i in range(50):
        print(i)
        
        # for model_type in model_types:
            # 4) Initialize the model
            # if model_type == "gru":
            #     
            # elif model_type == "lstm":
        # model = StockPriceLSTM(input_size, hidden_size, output_size)
        model = GRU3DClassifier(input_size, hidden_size, output_size, num_layers, dropout)
        model = model.to(device)

        # 5) Set up optimizer
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        print(f"Training {stock} | LR: {learning_rate} | Epochs: {num_epochs} "
              f"| Batch: {batch_size} | Security type: {security_type}")

        result = evaluate_rolling_unchanged_model_threshold(
            model=model,
            X=X_resampled,
            y=y_resampled,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            train_size=train_size,
            batch_size=batch_size,
            num_epochs=num_epochs,
            lower_threshold=0.5
        )

        # 7) Extract results
        rolling_predictions = result["rolling_predictions"]
        rolling_targets = result["rolling_targets"]
        test_accuracy = result["accuracy_nonzero"]
        loss_decrease_percentage = result["loss_decrease_percentage"]
        nonzero_preds = np.count_nonzero(rolling_predictions)
        final_train_loss = result["final_train_loss"]

        # 9) Create a record (dictionary) for this run
        run_record = {
            "STOCK": stock,
            "DATA_TYPE": security_type,
            "MODEL": model_type.upper(),
            "PROCESSING": processing,
            "ACCURACY": test_accuracy,
            "TRAIN_PCT_DECREASE": loss_decrease_percentage,
            "FINAL_TRAIN_LOSS": final_train_loss
        }

        # 10) Append to the results_list
        results_list.append(run_record)


----- LEARNING_RATE: 0.009883203468677031, SECURITY_TYPE: single_name, MODEL_TYPE: gru -----
0
Training AAPL | LR: 0.009883203468677031 | Epochs: 80 | Batch: 32 | Security type: single_name
[Epoch 5/80] Train Loss: 0.6937, Val Loss: 0.6931
[Epoch 10/80] Train Loss: 0.6937, Val Loss: 0.6927
[Epoch 15/80] Train Loss: 0.6934, Val Loss: 0.6921
[Epoch 20/80] Train Loss: 0.6928, Val Loss: 0.6912
[Epoch 25/80] Train Loss: 0.6916, Val Loss: 0.6873
[Epoch 30/80] Train Loss: 0.6910, Val Loss: 0.6877
[Epoch 35/80] Train Loss: 0.6913, Val Loss: 0.6889
[Epoch 40/80] Train Loss: 0.6898, Val Loss: 0.6874
[Epoch 45/80] Train Loss: 0.6882, Val Loss: 0.6874
[Epoch 50/80] Train Loss: 0.6892, Val Loss: 0.6883
[Epoch 55/80] Train Loss: 0.6884, Val Loss: 0.6899
[Epoch 60/80] Train Loss: 0.6878, Val Loss: 0.6840
[Epoch 65/80] Train Loss: 0.6843, Val Loss: 0.6827
[Epoch 70/80] Train Loss: 0.6842, Val Loss: 0.6835
[Epoch 75/80] Train Loss: 0.6835, Val Loss: 0.6789
[Epoch 80/80] Train Loss: 0.6807, Val Loss: 0

In [22]:
df = pd.DataFrame(results_list)
df

,STOCK,DATA_TYPE,MODEL,PROCESSING,ACCURACY,TRAIN_PCT_DECREASE,FINAL_TRAIN_LOSS
0,AAPL,options,LSTM,pca,0.511450,-93.096637,0.048020
1,AAPL,options,LSTM,pca,0.511450,-93.591413,0.044553
2,AAPL,options,LSTM,pca,0.473282,-97.815809,0.015188
3,AAPL,options,LSTM,pca,0.480916,-96.266795,0.025945
4,AAPL,options,LSTM,pca,0.603053,-96.150294,0.026757
...,...,...,...,...,...,...,...
95,AAPL,technical,LSTM,pca,0.579365,-37.621747,0.432957
96,AAPL,technical,LSTM,pca,0.515873,-83.982725,0.111194
97,AAPL,technical,LSTM,pca,0.420635,-30.648799,0.481398
98,AAPL,technical,LSTM,pca,0.547619,-80.392040,0.136304


In [24]:
output_folder = os.path.join(results_dir, f"individual_trials") 
df.to_csv(os.path.join(output_folder, f"lstm_pca_accuracy_bydatatype.csv"), index=False)

In [25]:
summary_stats = df.groupby(["MODEL", "DATA_TYPE"])["ACCURACY"].agg(["mean", "max", "min", "std"]).reset_index()
summary_stats.to_csv(os.path.join(output_folder, f"lstm_pca_bydatatype_summarystats.csv"), index=False)

In [58]:
output_folder = os.path.join(results_dir, 'individual_trials')
output_folder

'/home/jupyter-tfg2425paula/prediction_project_v3/02_results/individual_trials'

In [51]:
results_csv_path = 

if len(results_list) > 0:
    df = pd.DataFrame(results_list)

    if os.path.exists(results_csv_path):
        # Append without header
        df.to_csv(results_csv_path, mode='a', header=False, index=False)
    else:
        # Write new file with header
        df.to_csv(results_csv_path, index=False)

    # Clear the list before next iteration
    results_list = []

,STOCK,DATA_TYPE,MODEL,PROCESSING,ACCURACY,TRAIN_PCT_DECREASE,FINAL_TRAIN_LOSS
0,AAPL,single_name,GRU,clean,0.564885,-2.183248,0.680376
1,AAPL,single_name,GRU,clean,0.595420,-2.057334,0.681497
2,AAPL,single_name,GRU,clean,0.587786,-1.778138,0.683358
3,AAPL,single_name,GRU,clean,0.564885,-2.430913,0.678785
4,AAPL,single_name,GRU,clean,0.541985,-2.174941,0.680178
...,...,...,...,...,...,...,...
295,AAPL,technical,GRU,clean,0.468254,-78.074882,0.153156
296,AAPL,technical,GRU,clean,0.515873,-64.541400,0.246790
297,AAPL,technical,GRU,clean,0.500000,-62.280494,0.263006
298,AAPL,technical,GRU,clean,0.500000,-68.898970,0.216749
